# Classification des sons avec différentes valeurs de k

In [ ]:
# partie des imports
################################################################################################################

import matplotlib.pyplot as plt
import numpy as np 
import librosa
import librosa.display
import sounddevice as sd
import IPython.display as ipd
import glob
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [ ]:
verbose = False

In [ ]:
def Print(*args, **kwargs):
    '''méthode de print conditionnée par la valeur de la variable verbose'''
    global verbose
    try:
        if verbose:
            print(*args, *kwargs)
    except NameError:
        print(*args, *kwargs)
        
        
def build_bof(k, lesLabels, dimSons):
    '''méthode pour construire le Bag Of Features d'un échantillon de sons'''
    bofs = np.empty(shape = (0, k), dtype = int) # tableau des BoF
    i = 0 # indice pour le parcours de MFCC

    for nbS in dimSons: # Parcours des sons
        tmpBof = np.array([0]*k) # initialisation du BOF pour le son courant
        j = 0
        while j < nbS: # pour chaque MFCC de ce son
            tmpBof[lesLabels[i]] += 1
            j+=1
            i+=1
        tmpBof = tmpBof/nbS # transformation en pourcentage
        copyBof = tmpBof.copy()
        bofs = np.append(bofs, [copyBof], 0)
    
    return bofs


def build_MFCC(listSons, lesMfcc, dimSons, nb = 20):
    '''méthode pour construite le tableau des MFCC d'un échantillon de sons'''
    for s in listSons[:nb]: # on prends les nb premiers sons de la liste 1
        sig, rate = librosa.load(s)
        mfcc_feat = librosa.feature.mfcc(sig,rate,n_mfcc=13) # on calcule les MFCC du son

        Print('file: ', '/'.join(s.split('\\')[-2:]), '\tMFCC:', mfcc_feat.shape, '\tlen:', len(sig))

        lesMfcc = np.append(lesMfcc, mfcc_feat.transpose(), axis = 0) # on ajoute les MFCC du son à notre array
        dimSons.append(mfcc_feat.shape[1]) # et on "déclare" le nombre de MFCC du son
    return lesMfcc

In [ ]:
# partie d'initialisation des fichiers sons
################################################################################################################

basedir=r'C:\Users\Romain\Documents\SSII\TP noté'
class1 = 'four'
class2 = 'five'

listSons1 = glob.glob(basedir + '\\' + class1 + r'\*.wav')
listSons2 = glob.glob(basedir + '\\' + class2 + r'\*.wav')

Print(f'Classe 1: {len(listSons1)} sons.'.format())
Print(f'Classe 2: {len(listSons2)} sons.'.format())


# partie construction du vecteur de tous les MFCC
################################################################################################################

nb = 20 # nombre de sons dans l'échantillon
lesMfcc = np.empty(shape = (0, 13), dtype = float) # tableau de tous les MFCC (de tous les sons)
dimSons = []
lesMfcc = build_MFCC(listSons1, lesMfcc, dimSons, nb)
Print()
lesMfcc = build_MFCC(listSons2, lesMfcc, dimSons, nb)
Print('\nNombre total de MFCC:', lesMfcc.shape)
Print('Nombre de MFCC dans l\'ordre des sons:', dimSons)


nbTest=20
lesMfccTest = np.empty(shape=(0, 13), dtype=float)
dimSonsTest = []
# on va prendre les nb derniers sons
lesMfccTest = build_MFCC(listSons1[-nbTest:], lesMfccTest, dimSonsTest, nbTest)
lesMfccTest = build_MFCC(listSons2[-nbTest:], lesMfccTest, dimSonsTest, nbTest)
Print('shape :', lesMfccTest.shape)
Print(dimSonsTest)


# ici on va tester l'apprentissage en faisant varier le nombre de clusters k

# apprentissage
################################################################################################################

test_k_results = {}
train_k_results = {}

for k in range(2, 120):
    monKmeans = KMeans(n_clusters = k) # création de l'objet
    monKmeans.fit(lesMfcc) # apprentissage
    lesLabels = monKmeans.labels_ # tableau des numéros de "regroupement" de toutes nos MFCC
    
    bofs = build_bof(k, lesLabels, dimSons)
        
    logisticRegr = LogisticRegression() # création de l'objet de regression logistique

    # classes des sons (0 pour classe1 et 1 pour classe2)
    groundTruth = [0]*nb + [1]*nb

    logisticRegr.fit(bofs, groundTruth) # apprentissage
    resTrain = logisticRegr.predict(bofs)
    scoreTrain = f1_score(groundTruth, resTrain)
    print(f'train score pour k = {k}:', scoreTrain)
    train_k_results[k] = scoreTrain



    lesLabelsTest = monKmeans.predict(lesMfccTest) # prédiction des partitions pour ces MFCCs
    
    

    bofsTest = build_bof(k, lesLabelsTest, dimSonsTest)
    
    groundTruthTest = [0]*nbTest + [1]*nbTest
    resTest = logisticRegr.predict(bofsTest)
    scoreTest = f1_score(groundTruthTest,resTest)
    print(f'test score pour k = {k}:', scoreTest)
    test_k_results[k] = scoreTest

In [ ]:
# Courbe
if verbose:
    n = 16000
    fe = 8000
    te = 1/fe
    t = np.linspace(0, n*te, n,endpoint=False)
    s = 0.75*np.cos(2*np.pi*280*t) + 0.5*np.cos(2*np.pi*500*t)

    fig, graph = plt.subplots(figsize=(15, 5))
    graph.plot(list(test_k_results.keys()), list(test_k_results.values()), label = 'Test')
    graph.plot(list(train_k_results.keys()), list(train_k_results.values()), label = 'Train')
    plt.legend()
    graph.set_xlabel('k')
    graph.set_ylabel('Taux de validité')
    graph.set_title('Pourcentage de validité en fonction de k')
    plt.show()

In [ ]:
k = max(test_k_results.values())
print(f"k max = {k}")

for nb in range(10, 301):
    lesMfcc = np.empty(shape = (0, 13), dtype = float) # tableau de tous les MFCC (de tous les sons)
    dimSons = []
    lesMfcc = build_MFCC(listSons1, lesMfcc, dimSons, nb)
    lesMfcc = build_MFCC(listSons2, lesMfcc, dimSons, nb)
    
    monKmeans = KMeans(n_clusters = k) # création de l'objet
    monKmeans.fit(lesMfcc) # apprentissage
    lesLabels = monKmeans.labels_ # tableau des numéros de "regroupement" de toutes nos MFCC
    
    bofs = build_bof(k, lesLabels, dimSons)
        
    logisticRegr = LogisticRegression() # création de l'objet de regression logistique

    # classes des sons (0 pour classe1 et 1 pour classe2)
    groundTruth = [0]*nb + [1]*nb

    logisticRegr.fit(bofs, groundTruth) # apprentissage
    resTrain = logisticRegr.predict(bofs)
    scoreTrain = f1_score(groundTruth, resTrain)
    print(f'train score pour k = {k}:', scoreTrain)
    train_k_results[k] = scoreTrain
    
    

    bofsTest = build_bof(k, lesLabelsTest, dimSonsTest)
    
    groundTruthTest = [0]*nbTest + [1]*nbTest
    resTest = logisticRegr.predict(bofsTest)
    scoreTest = f1_score(groundTruthTest,resTest)
    print(f'test score pour nb = {nb}:', scoreTest)